In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

In [ ]:
project_path = "/content/drive/My Drive/ASL_Gloss_Project"
os.makedirs(project_path, exist_ok=True) # Create if it doesn't exist
%cd "{project_path}"

/content/drive/My Drive/ASL_Gloss_Project


In [ ]:
!pwd

/content/drive/My Drive/ASL_Gloss_Project


accelerate is important for efficient training on GPUs, especially larger models.

In [ ]:
!pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 # For CUDA 11.8 (Colab's default)
!pip install -U transformers datasets accelerate pandas
!pip install sentencepiece # Required by T5 tokenizer
!pip install evaluate

Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.3/506.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully unins

In [ ]:
!ls /content/drive/MyDrive/ASL_Gloss_Project/gloss_model_generalized_output/best_model


ls: cannot access '/content/drive/MyDrive/ASL_Gloss_Project/gloss_model_generalized_output/best_model': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/ASL_Gloss_Project/

eng_asl_gloss.csv


ye vala h correct code


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
from datasets import Dataset, load_dataset
import pandas as pd
import os
import gdown

# --- Configuration ---

ASLG_PC12_PATH = "eng_asl_gloss.csv"
MODEL_NAME = "t5-small"
OUTPUT_DIR = "/content/drive/MyDrive/ASL_Gloss_Project/gloss_model_generalized_output"
BATCH_SIZE = 8
NUM_EPOCHS = 5
LEARNING_RATE = 3e-4
MAX_SEQ_LENGTH = 128

GOOGLE_DRIVE_FILE_ID = "1IZwGhRpGW6JhdPJ2wnoDktesNWenSbHZ" # Extracted from the link

# --- Step 1: Load and Prepare Dataset ---
def load_and_prepare_dataset(file_path):
    print(f"Loading data from {file_path}...")

    try:
        df = pd.read_csv(file_path)

        if 'english_text' not in df.columns or 'asl_gloss' not in df.columns:
            print("Error: CSV does not contain expected columns 'english_text' and 'asl_gloss'.")
            print(f"Found columns: {df.columns.tolist()}")
            return None

        dataset = Dataset.from_pandas(df)
        print(f"Successfully loaded {len(dataset)} examples from {file_path}.")
        print("First 5 examples:")
        print(dataset[:5])
        return dataset
    except FileNotFoundError:
        print(f"Error: Dataset file not found at {file_path}")
        print("Please ensure ASLG_PC12_PATH is correct and the file 'eng_asl_gloss.csv' exists in the current directory.")
        return None
    except Exception as e:
        print(f"Error loading dataset: {e}")
        print("Please check the format of your CSV file and ensure it's valid.")
        return None

# --- Step 2: Tokenization ---
def tokenize_function(examples):
    max_input_length = MAX_SEQ_LENGTH
    max_target_length = MAX_SEQ_LENGTH

    model_inputs = tokenizer(
        examples["english_text"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length"
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["asl_gloss"],
            max_length=max_target_length,
            truncation=True,
            padding="max_length"
        )

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Step 3: Training
def train_model(train_dataset, eval_dataset, tokenizer, model):
    print("\n--- Starting Model Training ---")

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    training_args = TrainingArguments(
        output_dir=OUTPUT_DIR,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        num_train_epochs=NUM_EPOCHS,
        learning_rate=LEARNING_RATE,
        save_strategy="epoch",
        save_total_limit=2,
        eval_strategy="epoch",
        logging_dir="./logs",
        logging_steps=500,
        report_to="none",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        fp16=True,
        gradient_accumulation_steps=4,
        dataloader_num_workers=2,
        optim="adamw_torch",
        group_by_length=True,
        warmup_ratio=0.1,
        disable_tqdm=False,
    )
    model.gradient_checkpointing_enable()

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
    )

    # Check for existing checkpoints to resume training
    latest_checkpoint = None
    if os.path.exists(OUTPUT_DIR):
        checkpoints = [d for d in os.listdir(OUTPUT_DIR) if os.path.isdir(os.path.join(OUTPUT_DIR, d)) and d.startswith("checkpoint")]
        if checkpoints:
            # Sort checkpoints by epoch number to find the latest
            checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
            latest_checkpoint = os.path.join(OUTPUT_DIR, checkpoints[-1])
            print(f"Found latest checkpoint: {latest_checkpoint}. Resuming training...")

    trainer.train(resume_from_checkpoint=latest_checkpoint)
    print("\n--- Model Training Finished ---")

    # Save the best model separately after training
    model.save_pretrained(os.path.join(OUTPUT_DIR, "best_model"))
    tokenizer.save_pretrained(os.path.join(OUTPUT_DIR, "best_model"))
    print(f"Best model saved to {os.path.join(OUTPUT_DIR, 'best_model')}")

# --- Step 4: Inference ---
def generate_gloss(text, tokenizer, model, device):
    inputs = tokenizer(
        text,
        return_tensors="pt",
        max_length=MAX_SEQ_LENGTH,
        truncation=True,
        padding="max_length"
    ).to(device)

    outputs = model.generate(**inputs, max_new_tokens=MAX_SEQ_LENGTH, num_beams=5, early_stopping=True)
    gloss = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return gloss

# --- Main execution ---
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    print(f"Loading tokenizer and model: {MODEL_NAME}")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME).to(device)
    # model.eval() # No need to set to eval mode before training

    # Download the dataset from Google Drive
    print(f"Downloading dataset from Google Drive (ID: {GOOGLE_DRIVE_FILE_ID}) to {ASLG_PC12_PATH}...")
    try:
        gdown.download(id=GOOGLE_DRIVE_FILE_ID, output=ASLG_PC12_PATH, quiet=False)
        print("Download complete.")
    except Exception as e:
        print(f"Error downloading file: {e}")
        # Exit if download fails
        exit()


    raw_dataset = load_and_prepare_dataset(ASLG_PC12_PATH)

    # Check if dataset loaded successfully
    if raw_dataset is None:
        print("Exiting due to dataset loading error.")
        exit()

    split_dataset = raw_dataset.train_test_split(test_size=0.1, seed=42)
    train_dataset = split_dataset["train"]
    eval_dataset = split_dataset["test"]

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(eval_dataset)}")

    print("Tokenizing datasets...")
    tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['english_text', 'asl_gloss'])
    tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True, remove_columns=['english_text', 'asl_gloss'])

    train_model(tokenized_train_dataset, tokenized_eval_dataset, tokenizer, model)

    print("\n--- Testing Trained Model ---")
    loaded_tokenizer = AutoTokenizer.from_pretrained(os.path.join(OUTPUT_DIR, "best_model"))
    loaded_model = AutoModelForSeq2SeqLM.from_pretrained(os.path.join(OUTPUT_DIR, "best_model")).to(device)
    loaded_model.eval()

    test_texts = [
        "I am going home today.",
        "Hello, how are you feeling?",
        "Thank you for your help.",
        "What is your name, please?",
        "I want to drink some water.",
        "She is a very happy person.",
        "I do not understand what you mean.",
        "Tomorrow I have to work.",
        "He wants to eat food with his family.",
        "Can you help me with this problem?",
        "Where is the nearest hospital?",
        "The boy is playing with his toy car.",
        "Did you see that movie yesterday?"
    ]

    for text in test_texts:
        predicted_gloss = generate_gloss(text, loaded_tokenizer, loaded_model, device)
        print(f"English: \"{text}\" -> Predicted Gloss: \"{predicted_gloss}\"")

Using device: cuda
Loading tokenizer and model: t5-small


Downloading...
From: https://drive.google.com/uc?id=1IZwGhRpGW6JhdPJ2wnoDktesNWenSbHZ
To: /content/drive/MyDrive/ASL_Gloss_Project/eng_asl_gloss.csv
100%|██████████| 13.4M/13.4M [00:00<00:00, 181MB/s]


Download complete.
Loading data from eng_asl_gloss.csv...
Successfully loaded 87710 examples from eng_asl_gloss.csv.
First 5 examples:
{'english_text': ['\ufeffmembership of parliament see minutes\n', 'approval of minutes of previous sitting see minutes\n', 'membership of parliament see minutes\n', 'verification of credentials see minutes\n', 'documents received see minutes\n'], 'asl_gloss': ['\ufeffMEMBERSHIP PARLIAMENT SEE MINUTE\n', 'APPROVAL MINUTE DESC-PREVIOUS SIT SEE MINUTE\n', 'MEMBERSHIP PARLIAMENT SEE MINUTE\n', 'VERIFICATION CREDENTIALS SEE MINUTE\n', 'DOCUMENT RECEIVE SEE MINUTE\n']}
Train dataset size: 78939
Validation dataset size: 8771
Tokenizing datasets...


Map:   0%|          | 0/78939 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


...

/tmp/ipython-input-477163132.py:101: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.102900,0.038908
2,0.046300,0.019848
3,0.030300,0.014106
4,0.022900,0.011752


Epoch,Training Loss,Validation Loss
1,0.102900,0.038908
2,0.046300,0.019848
3,0.030300,0.014106
4,0.022900,0.011752
5,0.020900,0.011105


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].



--- Model Training Finished ---
Best model saved to /content/drive/MyDrive/ASL_Gloss_Project/gloss_model_generalized_output/best_model

--- Testing Trained Model ---
English: "I am going home today." -> Predicted Gloss: "X-I BE GO HOME TODAY."
English: "Hello, how are you feeling?" -> Predicted Gloss: "HELL, HOW BE X-YOU FEELING?"
English: "Thank you for your help." -> Predicted Gloss: "THANK X-YOU FOR X-YOU HELP."
English: "What is your name, please?" -> Predicted Gloss: "WHAT BE X-YOU NAME, PLEASE?"
English: "I want to drink some water." -> Predicted Gloss: "X-I WANT TO DRINK SOME WATER."
English: "She is a very happy person." -> Predicted Gloss: "SHE BE DESC-VERY DESC-HAPPY PERSON."
English: "I do not understand what you mean." -> Predicted Gloss: "X-I DO DESC-NOT UNDERSTAND WHAT X-YOU MEAN."
English: "Tomorrow I have to work." -> Predicted Gloss: "MORROW X-I HAVE TO WORK."
English: "He wants to eat food with his family." -> Predicted Gloss: "X-HE WANT TO EAT FOOD WITH X-HE FAMILY.

In [ ]:
import os
print("Model saved at:", os.path.join(OUTPUT_DIR, "best_model"))